## Ghép Biểu Tượng Nhà Vệ Sinh

### 1. Mô tả bài toán

Nhiệm vụ này yêu cầu huấn luyện một mô hình để học mối quan hệ giữa **biểu tượng nam và nữ của cùng một nhà vệ sinh**. Dựa trên dữ liệu huấn luyện đã được gán nhãn, bạn cần huấn luyện một mô hình ghép sao cho, với một ảnh truy vấn, mô hình sẽ tìm được **biểu tượng khác giới** (ví dụ: ghép biểu tượng nam trong ảnh đã cắt với biểu tượng nữ trong ảnh gốc), với điều kiện cả hai biểu tượng đều thuộc **cùng một nhà vệ sinh**.

Ví dụ, với một ảnh truy vấn (biểu tượng nam đã cắt) như Hình 1, đối tượng cần ghép là biểu tượng nữ trong ảnh gốc thuộc cùng nhà vệ sinh như Hình 2.

| <img src="../IOAI-2025-main\IOAI-2025-main\Individual-Contest\Restroom\figs\Restroom Fig 1.png" height="200"> | <img src="../IOAI-2025-main\IOAI-2025-main\Individual-Contest\Restroom\figs\Restroom Fig 2.png" height="200"> |
| -------------------------------------------------- | -------------------------------------------------- |

---

### 2. Bộ dữ liệu

**(1) Tập huấn luyện:**

Dùng để huấn luyện mô hình. Cấu trúc thư mục:



In [ ]:
train/
├── crop/         # Biểu tượng đã cắt
│   ├── female/   1.png, 2.png, ...
│   └── male/
└── orig/         # Biểu tượng gốc
    ├── female/
    └── male/



Mỗi thư mục con chứa các biểu tượng được đặt tên từ `1.png` đến `82.png`, số thứ tự là ID của nhà vệ sinh. Với mỗi nhà vệ sinh, có bốn ảnh (bốn ảnh này có cùng ID ở tất cả các thư mục con):

- `crop/female/i.png` → Biểu tượng nữ đã cắt  
- `crop/male/i.png`  → Biểu tượng nam đã cắt  
- `orig/female/i.png` → Biểu tượng nữ gốc  
- `orig/male/i.png`  → Biểu tượng nam gốc  

**(2) Tập xác thực và tập kiểm tra:**

Tập xác thực (`test_a`) hoặc tập kiểm tra (`test_b`) gồm hai thư mục con:



In [ ]:
test_a/           (hoặc test_b/)
├── query/        # Biểu tượng đã cắt cần ghép
└── gallery/      # Tập hợp các biểu tượng gốc để ghép



- `query/`: các biểu tượng đã cắt cần ghép  
- `gallery/`: tập hợp các biểu tượng gốc để ghép  
- Lưu ý:
    - Không giống tập huấn luyện, tên file trong `query/` và `gallery/` được đánh số và xáo trộn độc lập, không thể ghép dựa vào ID
    - Với mỗi biểu tượng đã cắt trong `query/` sẽ có đúng hai biểu tượng gốc trong `gallery/` (một nam, một nữ cùng nhà vệ sinh), tức là $\text{len}(\text{gallery})=2 * \text{len}(\text{query})$
- Tất cả ảnh đều ở định dạng `.png`
- Tập xác thực có $10$ ảnh trong thư mục `query/`, tập kiểm tra có $30$ ảnh trong thư mục `query/`

---

### 3. Nhiệm vụ

Với mỗi ảnh trong thư mục `query/`, dự đoán ảnh trong `gallery/` mà:
- Là **khác giới** với ảnh truy vấn
- Thuộc **cùng nhà vệ sinh**

Việc ghép này phải được thực hiện **bằng mô hình đã huấn luyện**.

---

### 4. Yêu cầu nộp bài

Bạn phải nộp một notebook tên là `submission.ipynb`, gồm:

- Quá trình huấn luyện mô hình (dùng dữ liệu `train/`)
- Quá trình ghép cho cả hai tập kiểm tra (`test_a/` và `test_b/`)
- Notebook phải xuất ra hai file `.npy`:


In [ ]:
submission_a.npy     # Kết quả ghép cho tập xác thực (test_a)
submission_b.npy     # Kết quả ghép cho tập kiểm tra (test_b)



Mỗi file npy là một mảng một chiều, kích thước bằng số lượng truy vấn, mỗi giá trị là ID ảnh trong gallery.

Ví dụ một file `submission_a.npy` hợp lệ:

<img src="../IOAI-2025-main\IOAI-2025-main\Individual-Contest\Restroom\figs\Restroom Fig 3.png" width="300">

Bạn cũng có thể xem ví dụ output tại [baseline.ipynb](https://ioai.bohrium.com/notebooks/81153159178).

---

### 5. Chấm điểm

- Nếu bài nộp hoàn thành trong thời gian quy định, điểm được tính như sau:  
  $$
  \text{Score} = \frac{\text{Number of correct matches}}{\text{Total queries}}
  $$
  Điểm là số thực từ $0.0$ đến $1.0$ (bao gồm cả hai đầu).
- Nếu vượt quá thời gian, điểm sẽ là **0**.

---

### 6. Baseline và tập huấn luyện

- Có thể tham khảo giải pháp baseline bên dưới.
- Bộ dữ liệu nằm trong thư mục `training_set`.
- Điểm cao nhất của Ban Khoa học cho bài này là **0.90** trên Leaderboard B (dùng để chuẩn hóa điểm).
- Điểm baseline của Ban Khoa học cho bài này là **0.77** trên Leaderboard B (dùng để chuẩn hóa điểm).

In [1]:
from transformers.models.clip import CLIPConfig, CLIPModel, CLIPTextConfig, CLIPVisionConfig
# text_config = CLIPTextConfig()
# vision_config = CLIPVisionConfig()
config = CLIPConfig()
clip_model = CLIPModel(config)

c:\Users\Vitus\miniconda3\envs\mavclip\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers.models.clip.modeling_clip import clip_loss, _get_vector_norm
import torch
def contrastive_loss(image_embeds, text_embeds):
    image_embeds = image_embeds / _get_vector_norm(image_embeds)
    text_embeds = text_embeds / _get_vector_norm(text_embeds)

    # cosine similarity as logits
    logit_scale = torch.nn.Parameter(torch.tensor(2.6592))
    logits_per_text = torch.matmul(text_embeds, image_embeds.t().to(text_embeds.device))
    logits_per_text = logits_per_text * logit_scale.exp().to(text_embeds.device)

    logits_per_image = logits_per_text.t()
    loss = clip_loss(logits_per_text)
    return {
        'loss': loss,
        'logits': torch.argmax(logits_per_image, dim=1)
        }

In [3]:
image_embeds = torch.rand((2, 520))  # batch size >= 2
text_embeds = torch.rand((2, 520))
print(contrastive_loss(image_embeds, text_embeds))

{'loss': tensor(0.7974, grad_fn=<DivBackward0>), 'logits': tensor([1, 0])}


In [4]:
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
class MyEffecientNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.features = efficientnet_b0(weights=None)
    def forward(self, inputs):
        return self.features(inputs)

In [5]:
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.datasets import ImageFolder, VisionDataset
import os
from pytorch_lightning import seed_everything
from PIL import Image
from torchvision.tv_tensors import Image as IMG
# to_image = IMG()
seed_everything(42, True, verbose=False)

transform = EfficientNet_B0_Weights.DEFAULT.transforms()
class MyDatasets(VisionDataset):
    def __init__(self, root=None, transforms=None, transform=None, target_transform=None, query: str = 'male', gallery: str = 'female'):
        super().__init__(root, transforms, transform, target_transform)
        self.query = query
        self.gallery = gallery
        self.transform = transform

    def __getitem__(self, index):
        image_1 = Image.open(os.path.join(
            self.root, f"{self.query}/{index + 1}.png")).resize((224, 224))
        image_2 = Image.open(os.path.join(
            self.root, f"{self.gallery}/{index + 1}.png")).resize((224, 224))
        return {
            'query': {
                'data': self.transform(image_1),
                'label': index
            },
            
            'gallery': {
                'data': self.transform(image_2),
                'label': index
            }
        }

    def __len__(self):
        return len(os.listdir(os.path.join(self.root, self.gallery)))

dataset = MyDatasets(
    root=r"E:\Luyen-AIO\IOAI-2025-main\IOAI-2025-main\Individual-Contest\Restroom\training_set\crop", transform=transform)
print(len(dataset))
train_size = int(len(dataset) * 0.8)
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
print(val_dataset[0]['query'])
# plt.imshow(val_dataset[0]['query'].permute(1, 2, 0))
# plt.show()
# plt.imshow(val_dataset[0]['gallery'].permute(1, 2, 0))
# plt.show()

82
{'data': tensor([[[ 0.7591,  0.7419,  0.7248,  ...,  0.2796,  0.2967,  0.2967],
         [ 0.7591,  0.7419,  0.7248,  ...,  0.2967,  0.3138,  0.3138],
         [ 0.7419,  0.7419,  0.7248,  ...,  0.2796,  0.2967,  0.2967],
         ...,
         [ 0.1254,  0.1254,  0.1083,  ..., -0.0458, -0.0458, -0.0458],
         [ 0.1083,  0.1083,  0.0912,  ..., -0.0458, -0.0458, -0.0458],
         [ 0.0912,  0.0912,  0.0912,  ..., -0.0458, -0.0458, -0.0458]],

        [[ 0.9580,  0.9405,  0.9230,  ...,  0.4853,  0.4678,  0.4678],
         [ 0.9580,  0.9405,  0.9230,  ...,  0.5028,  0.4853,  0.4853],
         [ 0.9580,  0.9405,  0.9230,  ...,  0.5028,  0.5028,  0.5028],
         ...,
         [ 0.5028,  0.4853,  0.4678,  ...,  0.2752,  0.2927,  0.2927],
         [ 0.4853,  0.4678,  0.4503,  ...,  0.2752,  0.2752,  0.2752],
         [ 0.4678,  0.4678,  0.4503,  ...,  0.2752,  0.2752,  0.2752]],

        [[ 0.4265,  0.4091,  0.3916,  ..., -0.1835, -0.1835, -0.1661],
         [ 0.4265,  0.4091,  0.39

In [6]:
import pytorch_lightning as pl
from sklearn.metrics import accuracy_score
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import Adam
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
class WrapperModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.save_hyperparameters()
        self.model = MyEffecientNet()
        self.loss = contrastive_loss
    
    def forward(self, batch):
        return self.model(batch['query']['data']), self.model(batch['gallery']['data'])
    
    def training_step(self, batch, batch_idx):
        loss = self.loss(self(batch)[0], self(batch)[1])
        self.log("train_loss", loss['loss'], prog_bar=True)
        y_true = batch['gallery']['label']
        y_pred = loss['logits'].long().detach().cpu().numpy()
        acc = accuracy_score(y_true.cpu().numpy().flatten(), y_pred.flatten())
        self.log("train_acc", acc, on_epoch=True, on_step=False, prog_bar=True)
    
    def validation_step(self, batch, batch_idx):
        loss = self.loss(self(batch)[0], self(batch)[1])
        self.log("val_loss", loss['loss'], prog_bar=True)
        y_true = batch['gallery']['label']
        y_pred = loss['logits'].long().detach().cpu().numpy()
        acc = accuracy_score(y_true.cpu().numpy().flatten(), y_pred.flatten())
        self.log("val_acc", acc, on_epoch=True, on_step=False, prog_bar=True)

    def configure_optimizers(self):
        optimizer = Adam(self.parameters(), lr=1e-3)
        lr_scheduler = ReduceLROnPlateau(optimizer,mode='min', patience=5)
        return {
            'optimizer': optimizer,
            'lr_scheduler': {
                'scheduler': lr_scheduler,
                'monitor': 'val_loss'
            }
        }

trainer = Trainer(
    callbacks=[
        ModelCheckpoint(dirpath='checkpoints/restroom_effecientnetb0', filename='best', save_last=True, monitor='val_loss', mode='min'),
        EarlyStopping(mode='min', monitor='val_loss', patience=10)
    ],
    logger=[TensorBoardLogger(save_dir='tb_logs', name='restroom_effecientnetb0')],
    max_epochs=1000, gradient_clip_val=1.0, max_time="00:00:15:00", precision=32
)
model = WrapperModel()


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [7]:
train_dataloader = DataLoader(train_dataset, 36, True, num_workers=0)
val_dataloader = DataLoader(val_dataset, 36, True, num_workers=0)

In [8]:
# res = None
# for ele in train_dataloader:
#     print(ele['query'].dtype)
#     print(ele['query'].shape)
#     res = model(ele)
#     break

In [9]:
# res[0].shape

In [10]:
import warnings
warnings.filterwarnings('ignore')
trainer.fit(model, train_dataloader, val_dataloader)

You are using a CUDA device ('NVIDIA GeForce RTX 3050 Ti Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type           | Params | Mode 
-------------------------------------------------
0 | model | MyEffecientNet | 5.3 M  | train
-------------------------------------------------
5.3 M     Trainable params
0         Non-trainable params
5.3 M     Total params
21.154    Total estimated model params size (MB)
338       Modules in train mode
0         Modules in eval mode


Epoch 19: 100%|██████████| 2/2 [00:07<00:00,  0.26it/s, v_num=23, train_loss=3.550, val_loss=3.210, val_acc=0.000, train_acc=0.0154] 
